In [14]:
library(tidyverse)
library(repr)
options(repr.matrix.max.rows = 6)

In [15]:
pulsar_data <- read_csv("data/HTRU_2.csv")

Rows: 17897 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (9): 140.5625, 55.68378214, -0.234571412, -0.699648398, 3.199832776, 19....

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [16]:
colnames(pulsar_data) <- c("Mean of the integrated profile","Standard deviation of the integrated profile.",
                           "Excess kurtosis of the integrated profile.", "Skewness of the integrated profile.",
                           "Mean of the DM-SNR curve.", "Standard deviation of the DM-SNR curve.",
                           "Excess kurtosis of the DM-SNR curve", "Skewness of the DM-SNR curve.",
                           "Class")


In [4]:
pulsar_data

ERROR: Error in eval(expr, envir, enclos): object 'pulsar_data' not found


In [2]:
Test<-print("I cry")

[1] "I cry"
